In [1]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd

In [35]:
q='''select distinct t.name as tool, d.packageId as id
    from mavenAlert mA
    join dependency d on mA.dependencyId = d.id
    join tool t on mA.toolId = t.id'''
fullDf= sql.pd_read_sql(q)
deps = []
tools = []
#deps[i] will be the dep list for tool[i]
for tool, df in fullDf.groupby('tool'):
    tools.append(tool)
    s = set(df.id)
    deps.append(s)
arr= [[0 for x in range(len(tools))] for x in range(len(tools))]
for i in range(len(arr)):
    arr[i][i]=100.0
for i in range(len(tools)):
    for j in range(i+1, len(tools)):
        #how many of i's deps is common with j's?
        total = len(deps[i])
        common = total - len(deps[i]-deps[j])
        ratio = round(common/total * 100,1)
        arr[i][j] = ratio

        #how many of j's deps is common with i's
        total = len(deps[j])
        common = total - len(deps[j]-deps[i])
        ratio = round(common/total * 100,1)
        arr[j][i] = ratio
arr


[[100.0, 40.0, 95.0, 95.0, 100.0, 75.0, 85.0],
 [22.2, 100.0, 61.1, 75.0, 94.4, 97.2, 38.9],
 [5.7, 6.6, 100.0, 17.5, 19.3, 15.4, 12.0],
 [19.8, 28.1, 60.4, 100.0, 88.5, 60.4, 54.2],
 [18.7, 31.8, 59.8, 79.4, 100.0, 61.7, 50.5],
 [16.5, 38.5, 56.0, 63.7, 72.5, 100.0, 41.8],
 [22.4, 18.4, 52.6, 68.4, 71.1, 50.0, 100.0]]

In [36]:
print('&'.join(tools)) 
print('\\\\')
for i in range(len(arr)):
    temp = [tools[i]]
    for j in range(len(arr[i])):
        temp.append(str(arr[i][j]) + '\\%')
    print('&'.join(temp))
    print('\\\\')
    

    

Github Dependabot&amp;Maven Security Versions&amp;OWASP Dependency-Check&amp;Snyk&amp;SourceClear&amp;Steady&amp;WhiteSource
\\
Github Dependabot&amp;100.0\%&amp;40.0\%&amp;95.0\%&amp;95.0\%&amp;100.0\%&amp;75.0\%&amp;85.0\%
\\
Maven Security Versions&amp;22.2\%&amp;100.0\%&amp;61.1\%&amp;75.0\%&amp;94.4\%&amp;97.2\%&amp;38.9\%
\\
OWASP Dependency-Check&amp;5.7\%&amp;6.6\%&amp;100.0\%&amp;17.5\%&amp;19.3\%&amp;15.4\%&amp;12.0\%
\\
Snyk&amp;19.8\%&amp;28.1\%&amp;60.4\%&amp;100.0\%&amp;88.5\%&amp;60.4\%&amp;54.2\%
\\
SourceClear&amp;18.7\%&amp;31.8\%&amp;59.8\%&amp;79.4\%&amp;100.0\%&amp;61.7\%&amp;50.5\%
\\
Steady&amp;16.5\%&amp;38.5\%&amp;56.0\%&amp;63.7\%&amp;72.5\%&amp;100.0\%&amp;41.8\%
\\
WhiteSource&amp;22.4\%&amp;18.4\%&amp;52.6\%&amp;68.4\%&amp;71.1\%&amp;50.0\%&amp;100.0\%
\\


In [39]:
q='select packageId from contrast'
df= sql.pd_read_sql(q)
s=set(df.packageId)
for i in range(len(tools)):
    total = len(s)
    common = total - (len(s-deps[i]))
    print(common/total)

0.35294117647058826
0.29411764705882354
0.7647058823529411
0.9411764705882353
0.9411764705882353
0.7647058823529411
0.7647058823529411


In [44]:
q='select packageId from contrast'
df= sql.pd_read_sql(q)
s=set(df.packageId)
for i in range(len(tools)):
    print(tools[i])
    total = len(deps[i])
    common = total - (len(deps[i]-s))
    print(common/total)

Github Dependabot
0.3
Maven Security Versions
0.1388888888888889
OWASP Dependency-Check
0.0391566265060241
Snyk
0.16666666666666666
SourceClear
0.14953271028037382
Steady
0.14285714285714285
WhiteSource
0.17105263157894737


In [28]:
q='''select distinct t.name as tool, d.packageId as id
    from npmAlert nA
    join dependency d on nA.dependencyId = d.id
    join tool t on nA.toolId = t.id'''
fullDf= sql.pd_read_sql(q)

deps = []
tools = []
#deps[i] will be the dep list for tool[i]
for tool, df in fullDf.groupby('tool'):
    tools.append(tool)
    s = set(df.id)
    deps.append(s)
arr= [[0 for x in range(len(tools))] for x in range(len(tools))]
for i in range(len(arr)):
    arr[i][i]=100.0
for i in range(len(tools)):
    for j in range(i+1, len(tools)):
        #how many of i's deps is common with j's?
        total = len(deps[i])
        common = total - len(deps[i]-deps[j])
        ratio = round(common/total * 100,1)
        arr[i][j] = ratio

        #how many of j's deps is common with i's
        total = len(deps[j])
        common = total - len(deps[j]-deps[i])
        ratio = round(common/total * 100,1)
        arr[j][i] = ratio
arr

[[100.0, 25.0, 37.5, 40.6, 50.0],
 [13.8, 100.0, 82.8, 63.8, 62.1],
 [5.0, 20.1, 100.0, 31.8, 30.5],
 [14.4, 41.1, 84.4, 100.0, 86.7],
 [18.0, 40.4, 82.0, 87.6, 100.0]]

In [29]:
print('&'.join(tools)) 
print('\\\\')
for i in range(len(arr)):
    temp = [tools[i]]
    for j in range(len(arr[i])):
        temp.append(str(arr[i][j]) + '\\%')
    print('&'.join(temp))
    print('\\\\')
    

Github Dependabot&amp;NPM Audit&amp;OWASP Dependency-Check&amp;Snyk&amp;WhiteSource
\\
Github Dependabot&amp;100.0\%&amp;25.0\%&amp;37.5\%&amp;40.6\%&amp;50.0\%
\\
NPM Audit&amp;13.8\%&amp;100.0\%&amp;82.8\%&amp;63.8\%&amp;62.1\%
\\
OWASP Dependency-Check&amp;5.0\%&amp;20.1\%&amp;100.0\%&amp;31.8\%&amp;30.5\%
\\
Snyk&amp;14.4\%&amp;41.1\%&amp;84.4\%&amp;100.0\%&amp;86.7\%
\\
WhiteSource&amp;18.0\%&amp;40.4\%&amp;82.0\%&amp;87.6\%&amp;100.0\%
\\
